<a href="https://colab.research.google.com/github/SSenitha/CCS3052_Advance_DSA/blob/Sandaru-s-Branch/adjacencyMatrix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Load data from the repository
!wget https://raw.githubusercontent.com/SSenitha/CCS3052_Advance_DSA/refs/heads/main/Cities_of_SriLanka.csv

--2025-09-09 18:28:18--  https://raw.githubusercontent.com/SSenitha/CCS3052_Advance_DSA/refs/heads/main/Cities_of_SriLanka.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 264501 (258K) [text/plain]
Saving to: ‘Cities_of_SriLanka.csv’

Cities_of_SriLanka. 100%[===================>] 258.30K  --.-KB/s    in 0.04s   

2025-09-09 18:28:18 (6.91 MB/s) - ‘Cities_of_SriLanka.csv’ saved [264501/264501]



In [2]:
#Import necessary libraries

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import stats


In [6]:
# Read the dataset and output the total count
df = pd.read_csv('/content/Cities_of_SriLanka.csv')
print(f"Total rows count: {df['city id'].count()}")
df.head()

Total rows count: 2155


,city id,district_id,name_en,name_si,name_ta,sub_name_en,sub_name_si,sub_name_ta,postcode,latitude,longitude
0,1,1,Akkaraipattu,අක්කරපත්තුව,அக்கரைப்பற்று,NULL,NULL,NULL,32400.0,7.218428,81.854116
1,2,1,Ambagahawatta,අඹගහවත්ත,அம்பகஹவத்த,NULL,NULL,NULL,90326.0,7.301756,81.674729
2,3,1,Ampara,අම්පාර,அம்பாறை,NULL,NULL,NULL,32000.0,7.301756,81.674729
3,4,1,Bakmitiyawa,බක්මිටියාව,பக்மிடியாவ,NULL,NULL,NULL,32024.0,7.029632,81.680205
4,5,1,Deegawapiya,දීඝවාපිය,தீகவாபி,NULL,NULL,NULL,32006.0,7.301756,81.674729
